# PyTorch를 사용한 MNIST 훈련

<div class="alert alert-block alert-info">
⚠️ 이 노트북에서 작동하는 것으로 알려진 최신 SageMaker Distribution 이미지 버전은 <code>3.4.2</code>입니다. 다른 버전에서 문제가 발생하면 <code>3.4.2</code> 버전으로 다운그레이드하세요. <b>이를 위해서는 JupyterApp을 중지하고, SageMaker Distribution 이미지를 <code>3.4.2</code>로 다운그레이드한 후 JupyterLabApp을 다시 시작해야 변경사항이 적용됩니다</b>.</div>

In [ ]:
!pip uninstall -y torchvison
!pip install -qU torchvision

## 목차

1. [배경](#Background)
1. [설정](#Setup)
1. [데이터](#Data)
1. [훈련](#Train)
1. [호스팅](#Host)

---

## 배경

MNIST는 손글씨 숫자 분류에 널리 사용되는 데이터셋입니다. 손으로 쓴 숫자의 28x28 픽셀 회색조 이미지 70,000개에 레이블이 지정되어 있습니다. 데이터셋은 60,000개의 훈련 이미지와 10,000개의 테스트 이미지로 분할됩니다. 10개의 클래스가 있습니다(10개 숫자 각각에 대해 하나씩). 이 튜토리얼은 PyTorch를 사용하여 SageMaker에서 MNIST 모델을 훈련하고 테스트하는 방법을 보여줍니다.

SageMaker의 PyTorch에 대한 자세한 정보는 [sagemaker-pytorch-containers](https://github.com/aws/sagemaker-pytorch-containers)와 [sagemaker-python-sdk](https://github.com/aws/sagemaker-python-sdk) GitHub 리포지토리를 방문하세요.

---

## 설정

_이 노트북은 ml.m4.xlarge 노트북 인스턴스에서 생성되고 테스트되었습니다._

SageMaker 세션을 생성하고 다음을 지정하는 것부터 시작하겠습니다:

- 훈련 및 모델 데이터에 사용할 S3 버킷과 접두사. 이는 노트북 인스턴스, 훈련 및 호스팅과 동일한 리전에 있어야 합니다.
- 데이터에 대한 훈련 및 호스팅 액세스 권한을 부여하는 데 사용되는 IAM 역할 ARN. 이를 생성하는 방법에 대한 설명서를 참조하세요. 노트북 인스턴스, 훈련 및/또는 호스팅에 두 개 이상의 역할이 필요한 경우 `sagemaker.get_execution_role()`을 적절한 전체 IAM 역할 ARN 문자열로 바꿀어 주세요.


In [ ]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = "sagemaker/pytorch-mnist"

role = sagemaker.get_execution_role()

## 데이터
### 데이터 가져오기



In [ ]:
from torchvision.datasets import MNIST
from torchvision import transforms

MNIST.mirrors = ["https://sagemaker-sample-files.s3.amazonaws.com/datasets/image/MNIST/"]

MNIST(
    "lab03-pytorch-data",
    download=True,
    transform=transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
    ),
)

### S3에 데이터 업로드
`sagemaker.Session.upload_data` 함수를 사용하여 데이터셋을 S3 위치에 업로드합니다. 반환값 inputs는 위치를 식별하며, 훈련 작업을 시작할 때 나중에 사용합니다.


In [ ]:
inputs = sagemaker_session.upload_data(path="lab03-pytorch-data", bucket=bucket, key_prefix=prefix)
print("input spec (in this case, just an S3 path): {}".format(inputs))

## 훈련
### 훈련 스크립트
`mnist.py` 스크립트는 SageMaker 모델 훈련 및 호스팅에 필요한 모든 코드를 제공합니다(모델을 로드하는 `model_fn` 함수).
훈련 스크립트는 SageMaker 외부에서 실행할 수 있는 훈련 스크립트와 매우 유사하지만, 다음과 같은 다양한 환경 변수를 통해 훈련 환경에 대한 유용한 속성에 액세스할 수 있습니다:

* `SM_MODEL_DIR`: 모델 아티팩트를 작성할 디렉토리의 경로를 나타내는 문자열.
  이러한 아티팩트는 모델 호스팅을 위해 S3에 업로드됩니다.
* `SM_NUM_GPUS`: 현재 컨테이너에서 사용 가능한 GPU 수.
* `SM_CURRENT_HOST`: 컨테이너 네트워크에서 현재 컨테이너의 이름.
* `SM_HOSTS`: 모든 호스트를 포함하는 JSON 인코딩된 목록.

PyTorch 추정기의 `fit()` 메서드 호출에서 'training'이라는 하나의 입력 채널이 사용되었다고 가정하면, `SM_CHANNEL_[channel_name]` 형식에 따라 다음이 설정됩니다:

* `SM_CHANNEL_TRAINING`: 'training' 채널의 데이터가 포함된 디렉토리의 경로를 나타내는 문자열.

훈련 환경 변수에 대한 자세한 정보는 [SageMaker Containers](https://github.com/aws/sagemaker-containers)를 방문하세요.

일반적인 훈련 스크립트는 입력 채널에서 데이터를 로드하고, 하이퍼파라미터로 훈련을 구성하고, 모델을 훈련하고, 나중에 호스팅할 수 있도록 모델을 `model_dir`에 저장합니다. 하이퍼파라미터는 스크립트에 인수로 전달되며 `argparse.ArgumentParser` 인스턴스로 검색할 수 있습니다.

SageMaker가 훈련 스크립트를 가져오기 때문에, 이 예제에서와 같이 모델을 호스팅하는 데 동일한 스크립트를 사용하는 경우 훈련 코드를 메인 가드(``if __name__=='__main__':``) 안에 넣어야 합니다. 그래야 SageMaker가 실행의 잘못된 시점에서 실수로 훈련 코드를 실행하지 않습니다.

예를 들어, 이 노트북에서 실행하는 스크립트:

In [ ]:
!pygmentize mnist-pytorch.py

### SageMaker에서 훈련 실행

`PyTorch` 클래스를 사용하면 SageMaker 인프라에서 훈련 함수를 훈련 작업으로 실행할 수 있습니다. 훈련 스크립트, IAM 역할, 훈련 인스턴스 수, 훈련 인스턴스 유형 및 하이퍼파라미터로 구성해야 합니다. 이 경우 2개의 ```ml.c4.xlarge``` 인스턴스에서 훈련 작업을 실행합니다. 하지만 이 예제는 하나 또는 여러 개, CPU 또는 GPU 인스턴스에서 실행할 수 있습니다([사용 가능한 인스턴스 전체 목록](https://aws.amazon.com/sagemaker/pricing/instance-types/)). hyperparameters 매개변수는 훈련 스크립트에 전달될 값들의 딕셔너리입니다. 위의 `mnist-pytorch.py` 스크립트에서 이러한 값에 액세스하는 방법을 볼 수 있습니다.


In [ ]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
    entry_point="mnist-pytorch.py",
    role=role,
    py_version="py310",
    framework_version="2.2.0",
    instance_count=2,
    instance_type="ml.c5.2xlarge",
    hyperparameters={"epochs": 1, "backend": "gloo"},
)

`PyTorch` 객체를 구성한 후 S3에 업로드한 데이터를 사용하여 훈련할 수 있습니다. SageMaker는 데이터가 로컬 파일 시스템에서 사용 가능하도록 보장하므로 훈련 스크립트는 단순히 디스크에서 데이터를 읽을 수 있습니다.


In [ ]:
estimator.fit({"training": inputs})

## 호스팅
### 엔드포인트 생성
훈련 후 `PyTorch` 추정기 객체를 사용하여 `PyTorchPredictor`를 구축하고 배포합니다. 이는 추론을 수행하는 데 사용할 수 있는 호스팅된 예측 서비스인 SageMaker 엔드포인트를 생성합니다.

위에서 언급했듯이 필요한 `mnist-pytorch.py` 스크립트에 `model_fn`의 구현이 있습니다. [sagemaker-pytorch-containers](https://github.com/aws/sagemaker-pytorch-containers)에 정의된 `input_fn`, `predict_fn`, `output_fn` 및 `transform_fm`의 기본 구현을 사용합니다.

deploy 함수의 인수를 사용하면 엔드포인트에 사용될 인스턴스의 수와 유형을 설정할 수 있습니다. 이는 훈련 작업에 사용한 값과 동일할 필요가 없습니다. 예를 들어, GPU 기반 인스턴스 세트에서 모델을 훈련한 다음 CPU 기반 인스턴스 플릿에 엔드포인트를 배포할 수 있지만, `mnist-pytorch.py`에서 한 것처럼 모델을 CPU 모델로 반환하거나 저장해야 합니다. 여기서는 모델을 단일 ```ml.m4.xlarge``` 인스턴스에 배포합니다.

In [ ]:
predictor = estimator.deploy(initial_instance_count=1, instance_type="ml.m4.xlarge")

### 평가

테스트 이미지를 사용하여 엔드포인트를 평가할 수 있습니다. 모델의 정확도는 얼마나 많이 훈련되었는지에 따라 달라집니다. 

In [ ]:
!ls lab03-pytorch-data/MNIST/raw

In [ ]:
import gzip
import numpy as np
import random
import os

data_dir = "lab03-pytorch-data/MNIST/raw"
with gzip.open(os.path.join(data_dir, "t10k-images-idx3-ubyte.gz"), "rb") as f:
    images = np.frombuffer(f.read(), np.uint8, offset=16).reshape(-1, 28, 28).astype(np.float32)


with gzip.open(os.path.join(data_dir, "t10k-labels-idx1-ubyte.gz"), "rb") as f:
    labels = np.frombuffer(f.read(), np.uint8, offset=8)

mask = random.sample(range(len(images)), 16)  # randomly select some of the test images
mask = np.array(mask, dtype=int)
data = images[mask]

true_labels = labels[mask]  # 선택된 이미지들의 정답 라벨

# 정답 확인 (추가)
print("선택된 이미지들의 실제 숫자:")
for i, label in enumerate(true_labels):
    print(f"이미지 {i}: {label}")

In [ ]:
response = predictor.predict(np.expand_dims(data, axis=1))
print("Raw prediction result:")
print(response)
print()

labeled_predictions = list(zip(range(10), response[0])) #response[x] 안에 값을 변경해보세요
print("Labeled predictions: ")
print(labeled_predictions)
print()

labeled_predictions.sort(key=lambda label_and_prob: 1.0 - label_and_prob[1])
print("Most likely answer: {}".format(labeled_predictions[0]))

### 정리

이 예제를 마친 후에는 예측 엔드포인트를 삭제하여 연결된 인스턴스를 해제하는 것을 잊지 마세요

In [ ]:
sagemaker_session.delete_endpoint(endpoint_name=predictor.endpoint_name)